# 1.Introduction

This projects is coming as a subproject for Kaggle's competetion: `Two Sigma: Using News to Predict Stock Movements`.
Basically its task is to predict if a given information/news brings positive or negative connotations.

This model may be/will be used for predicting how a given information coming from News data may affect a company's stock prices. 

# 2.Modules

## 2.1 NLTK modules for language processing 

In [83]:
import numpy as np
import pandas as pd
import random
import nltk
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
import re
from nltk.tokenize import word_tokenize, sent_tokenize, PunktSentenceTokenizer
from nltk.corpus import state_union, wordnet, stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer #Stemming words e.g Writing -> Write
"""Models for training and testing the results"""
from sklearn.naive_bayes import GaussianNB
#Keras - Deep LEarning models
from keras.layers.core import Dense,Flatten,Dropout
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import RMSprop

## 2.2 Downloading NLTK submodules

In [68]:
nltk.download('stopwords')
nltk.download('movie_reviews')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Łukasz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\Łukasz\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Łukasz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# 3. Deep text preprocessing

## 3.1 Uploading dataset from file

In [69]:
df = pd.read_csv('./data/data.tsv', delimiter = '\t')
df.head(5)

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


## 3.2 SentimentAnalyst class doing preprocessing step

In [70]:
class SentimentAnalyst:
    def __init__(self, df, feature_col, label_col):
        self.data = df
        self.corpora = None
        self.feature_col = feature_col
        self.label_col = label_col
    """Remove unnecessary symbols & lemmatization, stemming"""
    def preprocess_data(self):
        n_sentences = len(self.data)
        corpora = []
        ps = PorterStemmer()
        lemmatizer = WordNetLemmatizer()
        for review_it in range(0, n_sentences):
            review = re.sub('[^a-zA-Z]', ' ', self.data[self.feature_col][review_it])
            review = review.lower()
            review = review.split()
            stemmed = []
            for word in review:
                if not word in stopwords.words('english'):
                    word = lemmatizer.lemmatize(word)
                    stemmed.append(ps.stem(word))   
            review = stemmed
            review = ' '.join(review)
            corpora.append(review)
        self.corpora = corpora
        return corpora
    """Build vectors from words Bag of Words"""
    
    def vectorize(self, max_features):
        cv = CountVectorizer(max_features = max_features)
        X = cv.fit_transform(self.corpora).toarray()
        Y = df[self.label_col].values
        return X, Y

## 3.3 Analyst object

In [71]:
analyst = SentimentAnalyst(df, 'Review', "Liked")
analyst.preprocess_data()
X, Y = analyst.vectorize(600)

In [72]:
Y = to_categorical(Y) # Convertion into dummy variables

# 3.4 Splitting into train/test

In [73]:
X_train, X_test, Y_train, Y_test  = train_test_split(X, Y, test_size = 0.15, random_state = 42)

# 4. Building Deep Learning model (Artificial Neural Network)

In [92]:
model = Sequential([
    Dense(units = 32, activation = 'relu', input_dim = X_train.shape[1]),
    Dense(units = 64, activation = 'relu'),
    Dense(units = 128, activation = 'relu'),
    Dense(units = 64, activation = 'relu'),
    Dense(units = 2, activation = 'sigmoid'),
])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_61 (Dense)             (None, 32)                19232     
_________________________________________________________________
dense_62 (Dense)             (None, 64)                2112      
_________________________________________________________________
dense_63 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_64 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_65 (Dense)             (None, 2)                 130       
Total params: 38,050
Trainable params: 38,050
Non-trainable params: 0
_________________________________________________________________


In [93]:
optimizer = RMSprop(lr=0.0001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model.fit(X_train, Y_train, epochs = 500, validation_data = (X_test, Y_test))

Train on 850 samples, validate on 150 samples
Epoch 1/500
850/850 [==============================] - 0s 523us/step - loss: 0.6933 - acc: 0.5012 - val_loss: 0.6928 - val_acc: 0.5133
Epoch 2/500
850/850 [==============================] - 0s 77us/step - loss: 0.6924 - acc: 0.5565 - val_loss: 0.6926 - val_acc: 0.4933
Epoch 3/500
850/850 [==============================] - 0s 89us/step - loss: 0.6915 - acc: 0.5988 - val_loss: 0.6924 - val_acc: 0.5067
Epoch 4/500
850/850 [==============================] - 0s 75us/step - loss: 0.6906 - acc: 0.6294 - val_loss: 0.6922 - val_acc: 0.5067
Epoch 5/500
850/850 [==============================] - 0s 71us/step - loss: 0.6895 - acc: 0.6000 - val_loss: 0.6918 - val_acc: 0.5000
Epoch 6/500
850/850 [==============================] - 0s 85us/step - loss: 0.6881 - acc: 0.6000 - val_loss: 0.6913 - val_acc: 0.4933
Epoch 7/500
850/850 [==============================] - 0s 76us/step - loss: 0.6864 - acc: 0.6282 - val_loss: 0.6906 - val_acc: 0.4933
Epoch 8/500
850

850/850 [==============================] - 0s 80us/step - loss: 0.0865 - acc: 0.9729 - val_loss: 0.9114 - val_acc: 0.7267
Epoch 62/500
850/850 [==============================] - 0s 80us/step - loss: 0.0834 - acc: 0.9741 - val_loss: 0.9415 - val_acc: 0.7133
Epoch 63/500
850/850 [==============================] - 0s 71us/step - loss: 0.0796 - acc: 0.9753 - val_loss: 0.9488 - val_acc: 0.7267
Epoch 64/500
850/850 [==============================] - 0s 71us/step - loss: 0.0765 - acc: 0.9776 - val_loss: 0.9707 - val_acc: 0.7133
Epoch 65/500
850/850 [==============================] - 0s 72us/step - loss: 0.0735 - acc: 0.9765 - val_loss: 0.9879 - val_acc: 0.7133
Epoch 66/500
850/850 [==============================] - 0s 71us/step - loss: 0.0704 - acc: 0.9776 - val_loss: 1.0252 - val_acc: 0.7133
Epoch 67/500
850/850 [==============================] - 0s 80us/step - loss: 0.0675 - acc: 0.9788 - val_loss: 1.0164 - val_acc: 0.7133
Epoch 68/500
850/850 [==============================] - 0s 71us/step

850/850 [==============================] - 0s 75us/step - loss: 0.0163 - acc: 0.9929 - val_loss: 1.9292 - val_acc: 0.6933
Epoch 122/500
850/850 [==============================] - 0s 80us/step - loss: 0.0159 - acc: 0.9941 - val_loss: 1.9450 - val_acc: 0.6933
Epoch 123/500
850/850 [==============================] - 0s 71us/step - loss: 0.0158 - acc: 0.9929 - val_loss: 1.9532 - val_acc: 0.6933
Epoch 124/500
850/850 [==============================] - 0s 75us/step - loss: 0.0156 - acc: 0.9929 - val_loss: 1.9650 - val_acc: 0.6933
Epoch 125/500
850/850 [==============================] - 0s 71us/step - loss: 0.0156 - acc: 0.9941 - val_loss: 1.9973 - val_acc: 0.7067
Epoch 126/500
850/850 [==============================] - 0s 66us/step - loss: 0.0151 - acc: 0.9929 - val_loss: 2.0112 - val_acc: 0.7067
Epoch 127/500
850/850 [==============================] - 0s 71us/step - loss: 0.0149 - acc: 0.9929 - val_loss: 2.0441 - val_acc: 0.7067
Epoch 128/500
850/850 [==============================] - 0s 80

850/850 [==============================] - 0s 71us/step - loss: 0.0129 - acc: 0.9941 - val_loss: 2.5428 - val_acc: 0.7000
Epoch 182/500
850/850 [==============================] - 0s 71us/step - loss: 0.0122 - acc: 0.9941 - val_loss: 2.5185 - val_acc: 0.7000
Epoch 183/500
850/850 [==============================] - 0s 71us/step - loss: 0.0125 - acc: 0.9941 - val_loss: 2.5843 - val_acc: 0.7000
Epoch 184/500
850/850 [==============================] - 0s 75us/step - loss: 0.0124 - acc: 0.9941 - val_loss: 2.6082 - val_acc: 0.7000
Epoch 185/500
850/850 [==============================] - 0s 71us/step - loss: 0.0126 - acc: 0.9929 - val_loss: 2.5506 - val_acc: 0.7000
Epoch 186/500
850/850 [==============================] - 0s 85us/step - loss: 0.0124 - acc: 0.9929 - val_loss: 2.5429 - val_acc: 0.7000
Epoch 187/500
850/850 [==============================] - 0s 75us/step - loss: 0.0127 - acc: 0.9941 - val_loss: 2.5744 - val_acc: 0.7000
Epoch 188/500
850/850 [==============================] - 0s 75

850/850 [==============================] - 0s 80us/step - loss: 0.0128 - acc: 0.9941 - val_loss: 2.8231 - val_acc: 0.7200
Epoch 242/500
850/850 [==============================] - 0s 80us/step - loss: 0.0115 - acc: 0.9929 - val_loss: 2.8662 - val_acc: 0.7267
Epoch 243/500
850/850 [==============================] - 0s 80us/step - loss: 0.0124 - acc: 0.9929 - val_loss: 2.8357 - val_acc: 0.7333
Epoch 244/500
850/850 [==============================] - 0s 80us/step - loss: 0.0123 - acc: 0.9929 - val_loss: 2.8440 - val_acc: 0.7333
Epoch 245/500
850/850 [==============================] - 0s 80us/step - loss: 0.0125 - acc: 0.9941 - val_loss: 2.8296 - val_acc: 0.7200
Epoch 246/500
850/850 [==============================] - 0s 75us/step - loss: 0.0119 - acc: 0.9929 - val_loss: 2.8417 - val_acc: 0.7333
Epoch 247/500
850/850 [==============================] - 0s 80us/step - loss: 0.0122 - acc: 0.9929 - val_loss: 2.8440 - val_acc: 0.7333
Epoch 248/500
850/850 [==============================] - 0s 75

850/850 [==============================] - 0s 71us/step - loss: 0.0120 - acc: 0.9929 - val_loss: 2.9038 - val_acc: 0.7133
Epoch 302/500
850/850 [==============================] - 0s 71us/step - loss: 0.0125 - acc: 0.9929 - val_loss: 2.9051 - val_acc: 0.7267
Epoch 303/500
850/850 [==============================] - 0s 66us/step - loss: 0.0120 - acc: 0.9929 - val_loss: 2.9009 - val_acc: 0.7267
Epoch 304/500
850/850 [==============================] - 0s 75us/step - loss: 0.0122 - acc: 0.9941 - val_loss: 2.9331 - val_acc: 0.7267
Epoch 305/500
850/850 [==============================] - 0s 80us/step - loss: 0.0122 - acc: 0.9929 - val_loss: 2.9119 - val_acc: 0.7267
Epoch 306/500
850/850 [==============================] - 0s 75us/step - loss: 0.0129 - acc: 0.9941 - val_loss: 2.9343 - val_acc: 0.7267
Epoch 307/500
850/850 [==============================] - 0s 66us/step - loss: 0.0124 - acc: 0.9941 - val_loss: 2.8852 - val_acc: 0.7133
Epoch 308/500
850/850 [==============================] - 0s 75

850/850 [==============================] - 0s 75us/step - loss: 0.0118 - acc: 0.9918 - val_loss: 2.8845 - val_acc: 0.7200
Epoch 362/500
850/850 [==============================] - 0s 80us/step - loss: 0.0121 - acc: 0.9941 - val_loss: 2.8813 - val_acc: 0.7200
Epoch 363/500
850/850 [==============================] - 0s 80us/step - loss: 0.0122 - acc: 0.9929 - val_loss: 2.8758 - val_acc: 0.7200
Epoch 364/500
850/850 [==============================] - 0s 80us/step - loss: 0.0123 - acc: 0.9941 - val_loss: 2.9172 - val_acc: 0.7267
Epoch 365/500
850/850 [==============================] - 0s 71us/step - loss: 0.0123 - acc: 0.9929 - val_loss: 2.8973 - val_acc: 0.7267
Epoch 366/500
850/850 [==============================] - 0s 75us/step - loss: 0.0119 - acc: 0.9929 - val_loss: 2.8852 - val_acc: 0.7200
Epoch 367/500
850/850 [==============================] - 0s 80us/step - loss: 0.0127 - acc: 0.9941 - val_loss: 2.9062 - val_acc: 0.7267
Epoch 368/500
850/850 [==============================] - 0s 80

In [53]:
X_train = np.asarray(X_train)

In [66]:
X.shape

(1000, 599)